In [97]:
import math
import openai
import os
import sklearn.metrics
import random
import time
from langchain.llms import OpenAI

In [98]:
os.environ["OPENAI_API_KEY"] = "sk-xxxx"
openai.api_key = "sk-xxxx"
random.seed(0)

In [99]:
from task_utils import TASKS, load_data, load_prompt, generate_prompts

In [100]:
## your LLM stack goes here


# example of inference function for penai models
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
GPT_TURBO = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=600)

def call_llm_openai(prompt):
    output = GPT_TURBO([HumanMessage(content=prompt)])
    return output.content

# submission for finetuned openai model
import openai

def call_llm_finetuned_openai(prompt):
    output = openai.Completion.create(
    model="ada:ft-personal-2023-06-27-13-48-09",
    prompt=prompt,
    max_tokens=1,
    temperature=0.0,
    top_p=1.0,
    frequency_penalty=0.5,
   )
    return output.choices[0].text


# example of inference function for modal hosted models
import requests

def call_llm_modal(prompt):
    r = requests.post('https://xxxxxxxxx.modal.run', json={'question': prompt})
    output_dict = r.json()
    output = output_dict['output']
    completion = output[len(prompt):].strip()
    return completion


# example of inference function for baseten hosted models
import baseten
MODEL = baseten.deployed_model_version_id('xxxxx')

def call_llm_baseten(prompt):
    output = MODEL.predict({"prompt": prompt, "do_sample": True, "max_new_tokens": 300})
    completion = output['data']['generated_text'][len(prompt):].strip()
    return completion


call_llm = call_llm_finetuned_openai


In [101]:
def evaluate(tasks: list, tasks_dir: str):
    report = dict()
    for task in tasks:
        train_df, test_df = load_data(task=task, tasks_dir=tasks_dir)
        prompt_template = load_prompt(prompt_name="base_prompt.txt", task=task, tasks_dir=tasks_dir)
        prompts = generate_prompts(prompt_template=prompt_template, data_df=train_df)
        report[task] = dict()
        targets = list()
        outputs = list()
        print('task', task)
        for prompt, data in zip(prompts, train_df.iterrows()):
            datapoint_id, data = data
            output = call_llm(prompt)
            output = output.strip()
            targets.append(data['answer'])
            outputs.append(output)
            success = output == data['answer']
            report[task][datapoint_id] = {
                'prompt': prompt,
                'generated_output': output,
                'correct_output': data['answer'],
                'success': output == data['answer']
            }
        report[task]['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(targets, outputs)
        print('task balanced accuracy:', report[task]['balanced_accuracy'])
        print()
    
    print('Total Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))
    
    return report


In [102]:
tasks_dir = '../legalbench'

# warnings are to be expected

random.seed(0)
report = evaluate(tasks=random.sample(TASKS, 5), tasks_dir=tasks_dir)

INFO:openai:error_code=None error_message="This model's maximum context length is 2049 tokens, however you requested 2487 tokens (2486 in your prompt; 1 for the completion). Please reduce your prompt; or completion length." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False


task maud_change_in_law:__subject_to_"disproportionate_impact"_modifier


InvalidRequestError: This model's maximum context length is 2049 tokens, however you requested 2487 tokens (2486 in your prompt; 1 for the completion). Please reduce your prompt; or completion length.